In [2]:
!pip install transformers accelerate sentencepiece bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.4 MB/s eta 0:00:00


In [3]:
from huggingface_hub import login
login()

In [21]:
import torch
import subprocess
import tempfile
import textwrap
import ast
import re

DEBUG_HARMONIZE = True

In [5]:
# ============================================================
# Load All Models
# ============================================================

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


# ------------------------------------------------------------
# Code Generator (StarCoderBase-1B)
# ------------------------------------------------------------
GEN_MODEL = "Qwen/Qwen2.5-Coder-1.5B-Instruct"

gen_tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL)
gen_model = AutoModelForCausalLM.from_pretrained(
    GEN_MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
).to(device)

gen_pipe = pipeline(
    "text-generation",
    model=gen_model,
    tokenizer=gen_tokenizer,
    device=0 if device == "cuda" else -1
)


# ------------------------------------------------------------
# Syntax Critic (DeepSeek Coder 1.3B)
# ------------------------------------------------------------
SYNTAX_MODEL = "deepseek-ai/deepseek-coder-1.3b-instruct"

syntax_tokenizer = AutoTokenizer.from_pretrained(SYNTAX_MODEL)
syntax_model = AutoModelForCausalLM.from_pretrained(
    SYNTAX_MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
).to(device)

syntax_pipe = pipeline(
    "text-generation",
    model=syntax_model,
    tokenizer=syntax_tokenizer,
    device=0 if device == "cuda" else -1
)


# ------------------------------------------------------------
# Logic Critic (Phi-3 Mini)
# ------------------------------------------------------------
PHI3_MODEL = "microsoft/phi-3-mini-128k-instruct"

phi3_tokenizer = AutoTokenizer.from_pretrained(PHI3_MODEL)
phi3_model = AutoModelForCausalLM.from_pretrained(
    PHI3_MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
).to(device)

phi3_pipe = pipeline(
    "text-generation",
    model=phi3_model,
    tokenizer=phi3_tokenizer,
    device=0 if device == "cuda" else -1
)

print("All models loaded successfully.")

Device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0


All models loaded successfully.


In [6]:
def extract_single_function(code: str) -> str:
    """
    Extract exactly ONE function definition from the generated code.
    Removes repeated definitions and trailing garbage.
    """
    # Find all function blocks using a regex that captures indentation
    pattern = r"(def\s+[A-Za-z_]\w*\s*\([^)]*\)\s*:[\s\S]+?)(?=\ndef\s+|\Z)"
    matches = re.findall(pattern, code)

    if not matches:
        # If no function found, return sanitized code anyway
        return code.strip()

    # Return ONLY the first function block
    func = matches[0].strip()

    # Remove trailing blank lines
    func = func.rstrip()
    return func


In [26]:
# ============================================================
# Code Generator
# ============================================================

def generate_code(task_prompt, previous_code=None, feedback=None, max_new_tokens=200, test_list=None):
    prompt = "Task:\n" + task_prompt.strip() + "\n\n"

    if previous_code:
        prompt += "Previous attempt:\n" + previous_code.strip() + "\n\n"
    if feedback:
        prompt += "Feedback:\n" + feedback.strip() + "\n\n"

    prompt += """STRICT RULES:
1. Output ONLY valid Python code.
2. Do NOT include explanations, comments, docstrings, print statements, tests, or examples.
3. Do NOT repeat the function multiple times.
4. Generate exactly one function definition.
5. Do NOT wrap the code in markdown fences.
"""

    output = gen_pipe(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.4,
        top_p=0.9,
        repetition_penalty=1.05,
        pad_token_id=gen_tokenizer.eos_token_id,
    )[0]["generated_text"]

    if "Now output the code:" in output:
        output = output.split("Write only Python code:")[-1]

    output = output.replace("```python", "").replace("```", "")

    # Extract a SINGLE function
    output = extract_single_function(output)

    return output.strip()

In [8]:
def sanitize_unicode(text: str) -> str:
    """
    Remove problematic unicode characters such as arrows and non-ASCII symbols.
    Keeps only ASCII printable characters.
    """
    # Replace common arrow symbols
    text = text.replace("→", "->").replace("←", "<-")

    # Remove all non-ASCII characters
    text = text.encode("ascii", "ignore").decode()

    # Remove weird control characters (just in case)
    import re
    text = re.sub(r"[^\x09\x0A\x0D\x20-\x7E]", "", text)

    return text.strip()

In [9]:
# ============================================================
# Syntax Feedback (DeepSeek)
# ============================================================

def syntax_feedback(code: str, error_msg: str, window: int = 4) -> str:
    """
    Send only the minimal snippet around the syntax error.
    Force the model to output exactly ONE short Fix sentence.
    Strip out anything else aggressively.
    """

    # --- isolate the local snippet around error ---
    lines = code.splitlines()
    ln_match = re.search(r'line (\d+)', error_msg)
    if ln_match:
        ln = int(ln_match.group(1)) - 1
        start = max(0, ln - window)
        end = min(len(lines), ln + window + 1)
        snippet = "\n".join(lines[start:end])
    else:
        snippet = "\n".join(lines[:min(10, len(lines))])

    # --- extremely strict prompt ---
    prompt = f"""
Your task: Read the Python code and the SyntaxError message.

Output ONLY a very short correction hint, nothing else.
Do NOT rewrite the code. Do NOT repeat the code. Do NOT explain Python basics.

Your output must be LESS THAN 1–2 lines.

Now analyze:

<CODE>
{code}
</CODE>

<ERROR>
{error_msg}
</ERROR>
"""

    resp = syntax_pipe(
        prompt,
        max_new_tokens=80,
        do_sample=False,
        temperature=0.0,
        pad_token_id=syntax_tokenizer.eos_token_id
    )[0]["generated_text"].strip()

    # --- HARD CLEANUP --- (Updated for stricter extraction)
    # Attempt to find the actual fix statement, ignoring preamble and examples
    feedback_start_idx = -1
    if "Fix:" in resp:
        # Find the last occurrence of "Fix:" to avoid matching within model's examples
        feedback_start_idx = resp.rfind("Fix:")
    elif "Reason:" in resp:
        # Fallback if no explicit "Fix:" is given, look for "Reason:"
        feedback_start_idx = resp.rfind("Reason:")

    if feedback_start_idx != -1:
        resp = resp[feedback_start_idx:]
        # Ensure it starts with 'Fix: ' for consistency if it was 'Reason:'
        if not resp.startswith("Fix:"):
            resp = "Fix: " + resp.replace("Reason:", "").strip()
    else:
        # If no clear 'Fix:' or 'Reason:', just take the first few lines and clean them
        resp_lines = resp.split('\n')
        resp = resp_lines[0]
        if len(resp_lines) > 1 and len(resp_lines[1]) < 80 and not resp_lines[1].strip().startswith('<'): # avoid picking up new prompt sections
            resp += ' ' + resp_lines[1].strip()
        resp = "Fix: " + resp.strip()

    # remove code blocks if DeepSeek tries to add them
    resp = re.sub(r"```.*?```", "", resp, flags=re.DOTALL)
    resp = resp.replace("Code Snippet:", "")
    resp = resp.replace("Code with error:", "")
    resp = resp.replace("Syntax error:", "")
    resp = resp.strip()

    # keep only first sentence
    resp = re.split(r'[.!?]', resp)[0]
    resp = "Fix: " + resp.replace("Fix:", "").strip()

    # remove stray quotes, backticks, unicode
    resp = resp.replace("`", "").replace("'", "").strip()
    resp = sanitize_unicode(resp)

    return resp

In [10]:
# ============================================================
# Syntax Checker
# ============================================================

def check_syntax(code):
    try:
        ast.parse(code)
        return None
    except SyntaxError as e:
        return f"SyntaxError: {e.msg} at line {e.lineno}, column {e.offset}"

In [11]:
# ============================================================
# Logic Feedback (Phi-3 Mini)
# ============================================================

def logic_feedback(code, error_output, test_list):
    test_text = "\n".join(test_list)

    prompt = f"""
You are a code debugging assistant.

Given:
1. The Python function written by a user.
2. A failing test case.
3. The exact runtime error or assertion error.

Your task:
Explain ONLY the root cause of the failure in ONE short paragraph.
Do NOT rewrite the code.
Do NOT fix the code.
Do NOT output new tests.
Do NOT restate the function or error message.

Output format:
Reason: <1 short precise explanation>

Now analyze:

<CODE>
{code}
</CODE>

<TEST>
{test_text}
</TEST>

<ERROR>
{error_output}
</ERROR>

"""

    resp = phi3_pipe(
        prompt,
        max_new_tokens=200,
        temperature=0.0,
        do_sample=False,
        pad_token_id=phi3_tokenizer.eos_token_id,
    )[0]["generated_text"]

    if "Fix:" in resp:
        resp = resp.split("Fix:", 1)[-1]

    return "Fix: " + resp.strip()

In [27]:
# --------------------------
# 2.1 Utility: Harmonize function names
# --------------------------
def harmonize_function_name(candidate_code: str, tests_code: str) -> str:
    """
    If the tests call a function with a specific name but the candidate
    defines a different top-level function name, rename the candidate's
    function to the expected name. This addresses the common MBPP case
    where a single top-level function is expected.
    """
    def_match = re.search(r'^\s*def\s+([A-Za-z_]\w*)\s*\(', candidate_code, flags=re.MULTILINE)
    current_name = def_match.group(1) if def_match else None

    builtins_ignore = set([
        "assert", "print", "len", "range", "int", "str", "float", "list", "dict", "set", "tuple",
        "max", "min", "sum", "any", "all", "zip", "map", "filter", "open", "enumerate", "sorted",
        "reversed", "isinstance", "type", "input", "True", "False", "None", "pytest", "unittest"
    ])

    call_candidates = re.findall(r'\b([A-Za-z_]\w*)\s*\(', tests_code)
    desired_name = None
    for cname in call_candidates:
        if cname not in builtins_ignore:
            desired_name = cname
            break

    if not current_name or not desired_name or current_name == desired_name:
        return candidate_code

    candidate_code_modified = re.sub(
        r'(^\s*def\s+)' + re.escape(current_name) + r'(\s*\([^)]\)\s:)',
        r'\1' + desired_name + r'\2',
        candidate_code,
        flags=re.MULTILINE
    )

    candidate_code_modified = re.sub(
        r'(?<!def\s)' + r'\b' + re.escape(current_name) + r'\b',
        desired_name,
        candidate_code_modified
    )

    if DEBUG_HARMONIZE:
        print(f"[harmonize] Renamed function '{current_name}' -> '{desired_name}'")

    return candidate_code_modified

In [32]:

# ============================================================
# Run Tests (Logical Stage)
# ============================================================

def run_tests(code, test_list):
    test_block = "\n".join(test_list)
    full_script = code + "\n\n" + test_block
    harmonized_code = harmonize_function_name(code, full_script)
    with tempfile.NamedTemporaryFile(mode="w", suffix=".py", delete=False) as f:
        f.write(harmonized_code)
        temp_path = f.name

    try:
        result = subprocess.run(
            ["python3", temp_path],
            capture_output=True,
            text=True,
            timeout=5
        )
        return result.returncode, result.stdout, result.stderr
    except subprocess.TimeoutExpired:
        return 1, "", "Timeout during execution"

In [40]:
# ============================================================
# Main Refinement Loop
# ============================================================

def iterative_refinement(task_prompt, test_list, max_iters=20):
    code = None
    feedback = None

    for iteration in range(1, max_iters + 1):
        print("\nIteration", iteration)
        print("---------------")
        print("Task prompt:\n", task_prompt)
        print("---------------")
        print("Test list:\n", "\n".join(test_list))
        print("---------------")

        code = generate_candidate_code(task_prompt, reflection=feedback)
        print("Generated code:\n", code)

        # Harmonize function names
        code = harmonize_function_name(code, "\n".join(test_list))
        print("Harmonized code:\n", code)

        # Step 1: Syntax Stage
        error_msg = check_syntax(code)
        if error_msg:
            print("Syntax error:", error_msg)
            feedback = syntax_feedback(code, error_msg)
            print("Syntax feedback:", feedback)
            continue

        # Step 2: Logical Test Stage
        rc, out, err = run_tests(code, test_list)

        if rc == 0 and not err:
            print("All tests passed.")
            return True, code, iteration

        print("Logical error detected.")
        feedback = logic_feedback(code, err, test_list)
        print("Logical feedback:\n", feedback)

    print("Max iterations reached.")
    return False, code, max_iters

In [16]:
# EXAMPLE: Opposite signs

task = "Write a function opposite_signs(a, b) that returns True if a and b have opposite signs."

tests = [
    "assert opposite_signs(1, -1) == True",
    "assert opposite_signs(-5, 2) == True",
    "assert opposite_signs(5, 5) == False",
    "print('Tests passed')"
]

run_bol, final_code, iters = iterative_refinement(task, tests)
print("\nFinal code:")
print(final_code)
print("\nIterations:", iters)


Iteration 1
---------------
def opposite_signs(a, b):
    return a * b < 0

# Test the function with provided data points:
All tests passed.

Final code:
def opposite_signs(a, b):
    return a * b < 0

# Test the function with provided data points:

Iterations: 1


# MBPP dataset

In [17]:
from datasets import load_dataset

mbpp = load_dataset("mbpp", "sanitized", split="test")
print("MBPP tasks loaded:", len(mbpp))

README.md: 0.00B [00:00, ?B/s]

sanitized/train-00000-of-00001.parquet:   0%|          | 0.00/33.9k [00:00<?, ?B/s]

sanitized/test-00000-of-00001.parquet:   0%|          | 0.00/60.9k [00:00<?, ?B/s]

sanitized/validation-00000-of-00001.parq(…):   0%|          | 0.00/14.0k [00:00<?, ?B/s]

sanitized/prompt-00000-of-00001.parquet:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/257 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/43 [00:00<?, ? examples/s]

Generating prompt split:   0%|          | 0/7 [00:00<?, ? examples/s]

MBPP tasks loaded: 257


In [30]:
def run_mbpp_task(example, max_iters=20):
    task_prompt = example["prompt"]
    test_list = example["test_list"]

    print("\n==============================")
    print(f"Task ID: {example['task_id']}")
    print("Prompt:", task_prompt)
    print("==============================")

    success, code, iterations = iterative_refinement(task_prompt, test_list, max_iters=max_iters)

    return {"success": success, "code": code, "iterations": iterations}

In [41]:
subset = mbpp.select(range(25))

total = len(subset)
passed = 0

for i, ex in enumerate(subset):
    res = run_mbpp_task(ex, max_iters=3)
    if res["success"]:
        passed += 1
    print(f"Pass rate after {i+1}/{total}: {passed/(i+1):.3f}")


Task ID: 11
Prompt: Write a python function to remove first and last occurrence of a given character from the string.

Iteration 1
---------------
Task prompt:
 Write a python function to remove first and last occurrence of a given character from the string.
---------------
Test list:
 assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"
---------------
Generated code:
 def remove_char_occurrences(s, char):
    # Your code here
    return s

# Test the function with provided data points:
[harmonize] Renamed function 'remove_char_occurrences' -> 'remove_Occ'
Harmonized code:
 def remove_char_occurrences(s, char):
    # Your code here
    return s

# Test the function with provided data points:
All tests passed.
Pass rate after 1/25: 1.000

Task ID: 12
Prompt: Write a function to sort a given matrix in ascending order according to the sum of its rows.

Iteration 1
---------------
Task prompt:
 Write a function to sort a give